In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 152
USE = 'clicks'

PART = 'A'

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
USR = np.load(f'test_user_{PART}.npy')
len( USR )

900625

In [4]:
LOAD = 709
OFFSET = ''

df = pd.read_csv(f"../../data/candidate_scores/submission_v{LOAD}{OFFSET}.csv")
df = df.loc[df.session_type.str.contains(USE)]
df = cudf.DataFrame(df)
print( df.shape )
df.head()

(1801251, 2)


,session_type,labels
0,11098528_clicks,11830 588923 884502 1182614 571762 1157882 179...
1,11098529_clicks,1105029 1339838 1544564 1383767 459126 1694360...
2,11098530_clicks,409236 264500 1603001 254154 963957 167895 364...
3,11098531_clicks,396199 1271998 452188 1728212 1365569 624163 1...
4,11098532_clicks,876469 7651 1159379 108125 1202618 77906 17040...


In [5]:
OFFSET = '_wgt'

df2 = pd.read_csv(f"../../data/candidate_scores/submission_v{LOAD}{OFFSET}.csv")
df2 = df2.loc[df2.session_type.str.contains(USE)]
df2 = cudf.DataFrame(df2)
print( df2.shape )
df2.head()

(1801251, 2)


,session_type,labels
0,11098528_clicks,-1 14.5 12.485 12.469999999999999 11.99 11.98 ...
1,11098529_clicks,-1 9.5 8.49 8.48 7.995 7.975 6.97 6.45 6.295 5...
2,11098530_clicks,-1 -1 18.043499999999998 16.022 16.0205 16.004...
3,11098531_clicks,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 8.18865 7.866...
4,11098532_clicks,-1 -1 12.995000000000001 12.88 12.0 10.99 10.9...


In [6]:
df['user'] = df.session_type.str.partition('_')[0].astype('int32')
df = df.loc[df.user.isin(USR)].reset_index(drop=True)
df = df.drop('session_type',axis=1)
df.head()

,labels,user
0,11830 588923 884502 1182614 571762 1157882 179...,11098528
1,1105029 1339838 1544564 1383767 459126 1694360...,11098529
2,409236 264500 1603001 254154 963957 167895 364...,11098530
3,396199 1271998 452188 1728212 1365569 624163 1...,11098531
4,876469 7651 1159379 108125 1202618 77906 17040...,11098532


In [7]:
df2['user'] = df2.session_type.str.partition('_')[0].astype('int32')
df2 = df2.loc[df2.user.isin(USR)].reset_index(drop=True)
df2 = df2.drop('session_type',axis=1)
df2.head()

,labels,user
0,-1 14.5 12.485 12.469999999999999 11.99 11.98 ...,11098528
1,-1 9.5 8.49 8.48 7.995 7.975 6.97 6.45 6.295 5...,11098529
2,-1 -1 18.043499999999998 16.022 16.0205 16.004...,11098530
3,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 8.18865 7.866...,11098531
4,-1 -1 12.995000000000001 12.88 12.0 10.99 10.9...,11098532


In [8]:
tmp = df.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,item,pos
0,11830,0
0,588923,1
0,884502,2
0,1182614,3
0,571762,4


In [9]:
users = df[['user']].copy()
users.head()

,user
0,11098528
1,11098529
2,11098530
3,11098531
4,11098532


In [10]:
users = users.merge(tmp, left_index=True, right_index=True)

In [11]:
del df, tmp
gc.collect()

238

In [12]:
users.head()

,user,item,pos
66,11098594,778383,24
66,11098594,1597077,25
66,11098594,1176386,26
66,11098594,1250022,27
66,11098594,1629324,28


In [13]:
users.dtypes

user    int32
item    int32
pos     int32
dtype: object

In [14]:
tmp = df2.labels.str.split(' ').explode().astype('float32').rename('wgt').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,wgt,pos
0,-1.000,0
0,14.500,1
0,12.485,2
0,12.470,3
0,11.990,4


In [15]:
users2 = df2[['user']].copy()#.reset_index()
users2.head()

,user
0,11098528
1,11098529
2,11098530
3,11098531
4,11098532


In [16]:
del df2
gc.collect()

23

In [17]:
users2 = users2.merge(tmp, left_index=True, right_index=True)

In [18]:
users2.head()

,user,wgt,pos
91,11098619,1.740,52
91,11098619,1.735,53
91,11098619,1.710,54
91,11098619,1.690,55
91,11098619,1.225,56


In [19]:
users = users.merge(users2,on=['user','pos'],how='left')

In [20]:
users.head()

,user,item,pos,wgt
0,11098619,342897,84,0.605
1,11098619,653790,85,0.500
2,11098619,31490,86,0.500
3,11098619,1327137,87,0.500
4,11098619,1426624,88,0.500


In [21]:
del users2, tmp
gc.collect()

78

In [22]:
users.shape

(90062500, 4)

# Add Targets

In [23]:
test_labels = cudf.read_parquet('../../data/train_data/test_labels.parquet')
test_labels = test_labels.loc[test_labels['type']==USE]
print( test_labels.shape )
test_labels.head()

(1755534, 3)


,session,type,ground_truth
0,11098528,clicks,[1679529]
3,11098529,clicks,[1105029]
6,11098532,clicks,[1596491]
7,11098533,clicks,[1417450]
10,11098534,clicks,[908024]


In [24]:
tmp = test_labels.ground_truth.explode().astype('int32').rename('item')
tmp.head()

0     1679529
3     1105029
6     1596491
7     1417450
10     908024
Name: item, dtype: int32

In [25]:
test_users = test_labels[['session']].astype('int32').copy()
test_users.columns = ['user']

In [26]:
test_users = test_users.merge(tmp,left_index=True,right_index=True)

In [27]:
del test_labels, tmp
gc.collect()

50

In [28]:
test_users[USE] = np.int8(1)

In [29]:
test_users.dtypes

user      int32
item      int32
clicks     int8
dtype: object

In [30]:
test_users.head()

,user,item,clicks
14391,11110224,237004,1
14392,11110225,403215,1
14393,11110226,634452,1
14394,11110227,804558,1
14395,11110228,75686,1


In [31]:
users = users.merge(test_users, on=['user','item'], how='left').fillna(0)

In [32]:
del test_users
gc.collect()

40

In [33]:
users.head()

,user,item,pos,wgt,clicks
0,11098739,506879,4,10.495,0
1,11098739,259891,5,9.500,0
2,11098739,527968,6,9.460,0
3,11098739,1499305,7,7.990,0
4,11098739,253410,8,7.475,0


In [34]:
print(users.shape, users.dtypes)

(90062500, 5) user        int32
item        int32
pos         int32
wgt       float32
clicks       int8
dtype: object


In [35]:
users = users.to_pandas()

# Save

In [36]:
users.dtypes

user        int32
item        int32
pos         int32
wgt       float32
clicks       int8
dtype: object

In [37]:
users[USE] = users[USE].astype('int8')

In [38]:
users.to_parquet(f'../../data/train_with_features/train_v{VER}_{PART}.pqt',index=False)